In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')

In [ ]:
data = pd.read_csv(os.path.join('data', 'ibarraSoriaIntermittentExposureResults.csv'), thousands=',')

In [ ]:
data_control = data[['control' + str(_ + 1) for _ in range(6)]].copy(deep=True)
data_exposed = data[['exposed' + str(_ + 1) for _ in range(6)]].copy(deep=True)

In [ ]:
# normalize the data
total = np.mean(data_control.sum(axis=0))
for col_name in data_control.columns:
    data_control.loc[:, col_name] = data_control[col_name] / data_control[col_name].sum() * total
for col_name in data_exposed.columns:
    data_exposed.loc[:, col_name] = data_exposed[col_name] / data_exposed[col_name].sum() * total

In [ ]:
eps = np.spacing(1.0)

data_control_mean = data_control.mean(axis=1)
#data_control_mean = data_control.median(axis=1)
data_control_std = data_control.std(axis=1)/np.sqrt(6) # sem
data_control_std_rel = data_control_std / (data_control_mean + eps)

data_exposed_mean = data_exposed.mean(axis=1)
#data_exposed_mean = data_exposed.median(axis=1)
data_exposed_std = data_exposed.std(axis=1)/np.sqrt(6) # sem
data_exposed_std_rel = data_exposed_std / (data_exposed_mean + eps)

In [ ]:
from scipy.io import savemat

savemat(os.path.join('data', 'ibarra_soria_data.mat'),
        {'control_mean': data_control_mean.as_matrix(),
         'control_std': data_control_std.as_matrix(),
         'exposed_mean': data_exposed_mean.as_matrix(),
         'exposed_std': data_exposed_std.as_matrix(),
         'gene': data['gene'].as_matrix()})

In [ ]:
fold_change = (data_exposed_mean + eps) / (data_control_mean + eps)
fold_change_std_rel = data_control_std_rel + data_exposed_std_rel
fold_change_std = fold_change*fold_change_std_rel

log_fold_change = np.log2(fold_change)
log_fold_change_std = fold_change_std_rel

In [ ]:
plt.figure(figsize=(2.9, 2.3))

plt.errorbar(data_control_mean, log_fold_change, xerr=data_control_std, yerr=fold_change_std_rel,
             fmt='none', ecolor=[0.5, 0.5, 0.5], elinewidth=0.5)
plt.semilogx(data_control_mean, log_fold_change, '.', markersize=4)
plt.axhline(0, color='r', alpha=0.3)
plt.xlim(0.07, 3e4)
plt.ylim(-2.5, 2.5);

plt.xlabel('RNAseq counts, control', fontsize=8)
plt.ylabel('$\log_2$ fold change exposed/control', fontsize=8)

plt.savefig(os.path.join('figs', 'ibarra_soria_Fig5B_remake.pdf'), bbox_inches='tight')

#plt.semilogx(data_control_mean[442], log_fold_change[442], '.', color='r');

In [ ]:
#_ = plt.hist(np.log10(data_control_mean[data_control_mean > 0]), 50)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.hist(data_control_mean[data_control_mean > 0], bins=np.logspace(-1, 4, 50))
ax.set_xscale('log')
ax.set_xlabel('RNAseq counts')

# plt.savefig(os.path.join('figs', 'ibarra_soria_histogram.pdf'))

In [ ]:
fig = plt.figure(figsize=(7, 1.5))
ax = fig.add_subplot(111)
# np.random.seed(1243)
ax.fill_between(list(range(len(data_control_mean))), 0, data_control_mean)
ax.set_yscale('log')
ax.set_xlabel('Receptor index')
ax.set_ylabel('Copy number')
ax.set_xlim(0, len(data_control_mean))
sns.despine()
# ax.semilogy(data_control_mean)
# ax.bar(list(range(len(data_control_mean))), np.random.permutation(data_control_mean))
# sns.barplot(list(range(len(data_control_mean))), np.random.permutation(data_control_mean))

# plt.savefig(os.path.join('figs', 'receptor_distribution_ibarra_soria.svg', bbox_inches='tight'))

In [ ]:
# control_var_range = data_control.quantile([0.2, 0.8], axis=1).as_matrix()
# control_var_range = data_control.quantile([0.05, 0.95], axis=1).as_matrix()
control_var_range = data_control.quantile([0.25, 0.75], axis=1).as_matrix()
fig = plt.figure(figsize=(7, 1.5))
ax = fig.add_subplot(111)
np.random.seed(1243)
ax.fill_between(list(range(len(data_control_mean))), control_var_range[0], control_var_range[1], color='r', alpha=0.2)
ax.plot(data_control_mean, lw=0.5)
ax.set_yscale('log')
ax.set_xlabel('Receptor index')
ax.set_ylabel('Copy number')
ax.set_xlim(0, len(data_control_mean))
sns.despine()
# ax.semilogy(data_control_mean)

# plt.savefig('receptor_distribution_ibarra_soria_variation.svg', bbox_inches='tight')